In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from scipy.stats import chi2_contingency, pearsonr, spearmanr, kendalltau, shapiro, normaltest,f_oneway
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [48]:
melb = pd.read_csv('melb_data.csv')
melb.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


# Nomor 1 
- Tampilkan statistik deskriptif pada variabel Rooms, Distance, Landsize, dan Price!

In [49]:
# Display descriptive statistics for the specified columns
melb[['Rooms', 'Distance', 'Landsize', 'Price']].describe()

,Rooms,Distance,Landsize,Price
count,13580.000000,13580.000000,13580.000000,1.358000e+04
mean,2.937997,10.137776,558.416127,1.075684e+06
std,0.955748,5.868725,3990.669241,6.393107e+05
min,1.000000,0.000000,0.000000,8.500000e+04
25%,2.000000,6.100000,177.000000,6.500000e+05
50%,3.000000,9.200000,440.000000,9.030000e+05
75%,3.000000,13.000000,651.000000,1.330000e+06
max,10.000000,48.100000,433014.000000,9.000000e+06


# Nomor 2
- Berapa jumlah Rooms pada rumah yang harganya paling mahal?
- Berapa rata-rata harga rumah yang jarak ke CBD (variabel Distance) terdekat?
- Berapa rata-rata harga rumah yang luas tanahnya (variabel Landsize) 1 kali standar deviasi di atas rata-rata luas tanah?
- Berapa rentang (range) harga rumah (variabel Price) pada region Southern Metropolitan?

In [50]:
# 1. Berapa jumlah Rooms pada rumah yang harganya paling mahal?
max_price_idx = melb['Price'].idxmax()
rooms_max_price = melb.loc[max_price_idx, 'Rooms']
print(f"1. Jumlah Rooms pada rumah termahal: {rooms_max_price} rooms")
print(f"   Harga rumah termahal: ${melb['Price'].max()}\n")

# 2. Berapa rata-rata harga rumah yang jarak ke CBD (variabel Distance) terdekat?
min_distance = melb['Distance'].min()
closest_houses = melb[melb['Distance'] == min_distance]
avg_price_closest = closest_houses['Price'].mean()
print(f"2. Rata-rata harga rumah terdekat ke CBD: ${avg_price_closest}")
print(f"   Jarak terdekat: {min_distance} km\n")

# 3. Berapa rata-rata harga rumah yang luas tanahnya 1 kali standar deviasi di atas rata-rata luas tanah?
landsize_mean = melb['Landsize'].mean()
landsize_std = melb['Landsize'].std()
threshold_landsize = landsize_mean + landsize_std
large_landsize_houses = melb[melb['Landsize'] >= threshold_landsize]
avg_price_large_land = large_landsize_houses['Price'].mean()
print(f"3. Rata-rata harga rumah dengan luas tanah ≥ (mean + 1 std): ${avg_price_large_land}")
print(f"   Threshold luas tanah: {threshold_landsize} sqm")
print(f"   Jumlah rumah: {len(large_landsize_houses)} rumah\n")

# 4. Berapa rentang (range) harga rumah pada region Southern Metropolitan?
southern_metro = melb[melb['Regionname'] == 'Southern Metropolitan']
if len(southern_metro) > 0:
    min_price_sm = southern_metro['Price'].min()
    max_price_sm = southern_metro['Price'].max()
    price_range_sm = max_price_sm - min_price_sm
    print(f"4. Rentang harga rumah di Southern Metropolitan: ${price_range_sm}")
    print(f"   Harga minimum: ${min_price_sm}")
    print(f"   Harga maximum: ${max_price_sm}")
else:
    print("4. Region 'Southern Metropolitan' tidak ditemukan dalam data")

1. Jumlah Rooms pada rumah termahal: 3 rooms
   Harga rumah termahal: $9000000.0

2. Rata-rata harga rumah terdekat ke CBD: $671166.6666666666
   Jarak terdekat: 0.0 km

3. Rata-rata harga rumah dengan luas tanah ≥ (mean + 1 std): $880776.6666666666
   Threshold luas tanah: 4549.08536776588 sqm
   Jumlah rumah: 75 rumah

4. Rentang harga rumah di Southern Metropolitan: $7869000.0
   Harga minimum: $131000.0
   Harga maximum: $8000000.0


# Nomor 3
Nilai outlier adalah nilai yang berada di bawah Q1 - (1.5 * IQR) atau di atas Q3 + (1.5 * IQR).
- Hitung dulu Q1, Q2, dan IQR pada kolom Price (harga rumah)
- Hitung ada berapa nilai outlier pada kolom Price!

In [51]:
# Hitung Q1, Q2, Q3, dan IQR untuk kolom Price
Q1 = melb['Price'].quantile(0.25)
Q2 = melb['Price'].quantile(0.50)  # Median
Q3 = melb['Price'].quantile(0.75)
IQR = Q3 - Q1

# Hitung batas outlier
lower_bound = Q1 - (1.5 * IQR)
upper_bound = Q3 + (1.5 * IQR)

# Hitung jumlah outlier
outliers = melb[(melb['Price'] < lower_bound) | (melb['Price'] > upper_bound)]
num_outliers = len(outliers)

Q1, Q2, Q3, IQR, num_outliers

(np.float64(650000.0),
 np.float64(903000.0),
 np.float64(1330000.0),
 np.float64(680000.0),
 612)

# Nomor 4
Lakukan uji asumsi normalitas pada variabel harga rumah (Price) menggunakan:
1. Shapiro Wilk Test
2. D’Agostino and Pearson’s Test


In [52]:
# Shapiro Wilk Test
shapiro_stat, shapiro_p = shapiro(melb['Price'])

# D'Agostino and Pearson's Test
dagostino_stat, dagostino_p = normaltest(melb['Price'])

shapiro_stat, shapiro_p, dagostino_stat, dagostino_p

(np.float64(0.8328502932625694),
 np.float64(1.446571861512777e-79),
 np.float64(6894.345896172315),
 np.float64(0.0))

# Nomor 5
Jika diasumsikan telah memenuhi syarat statistik parametrik, hitung seberapa kuat korelasi variabel Rooms dengan Price, serta arah korelasinya!

In [53]:
# Hitung korelasi Pearson antara Rooms dan Price
correlation_coef, p_value = pearsonr(melb['Rooms'], melb['Price'])

print(f"Koefisien Korelasi Pearson: {correlation_coef}")
print(f"P-value: {p_value:.4e}")

# Interpretasi kekuatan korelasi
if abs(correlation_coef) < 0.3:
    strength = "lemah"
elif abs(correlation_coef) < 0.7:
    strength = "sedang"
else:
    strength = "kuat"

# Interpretasi arah korelasi
direction = "positif" if correlation_coef > 0 else "negatif"

print(f"\nInterpretasi:")
print(f"- Kekuatan korelasi: {strength}")
print(f"- Arah korelasi: {direction}")
print(f"- Artinya: Ada korelasi {strength} dan {direction} antara jumlah Rooms dengan Price")


pandas_corr = melb[['Rooms', 'Price']].corr()
pandas_corr

Koefisien Korelasi Pearson: 0.49663367618653004
P-value: 0.0000e+00

Interpretasi:
- Kekuatan korelasi: sedang
- Arah korelasi: positif
- Artinya: Ada korelasi sedang dan positif antara jumlah Rooms dengan Price


,Rooms,Price
Rooms,1.000000,0.496634
Price,0.496634,1.000000


# Nomor 6
- Dari beberapa Regionname, manakah Regionname yang variabel jumlah kamarnya (Rooms) memiliki korelasi terkuat dengan variabel harga rumah (Price)!
- Diasumsikan telah memenuhi syarat statistik parametrik.

In [54]:
# Korelasi Rooms vs Price untuk setiap region
correlations = {}
for region in melb['Regionname'].unique():
    region_data = melb[melb['Regionname'] == region]
    if len(region_data) > 2:
        corr, _ = pearsonr(region_data['Rooms'], region_data['Price'])
        correlations[region] = abs(corr)

# Region dengan korelasi terkuat
strongest_region = max(correlations, key=correlations.get)
strongest_corr = correlations[strongest_region]

print(f"Region dengan korelasi terkuat: {strongest_region}")
print(f"Koefisien korelasi: {strongest_corr}")

# Tampilkan top 3 region
sorted_corr = sorted(correlations.items(), key=lambda x: x[1], reverse=True)
print("\nTop 3 Region:")
for i, (region, corr) in enumerate(sorted_corr[:3], 1):
    print(f"{i}. {region}: {corr}")

Region dengan korelasi terkuat: Southern Metropolitan
Koefisien korelasi: 0.6837438462796436

Top 3 Region:
1. Southern Metropolitan: 0.6837438462796436
2. Western Victoria: 0.5518865952271457
3. Eastern Metropolitan: 0.5029928557468843


# Nomor 7
Jika diasumsikan tidak memenuhi syarat statistik parametrik, hitung seberapa kuat korelasi variabel Distance dengan Price, serta arah korelasinya!

In [55]:
# Hitung korelasi Spearman antara Distance dan Price
spearman_coef, spearman_p = spearmanr(melb['Distance'], melb['Price'])

print(f"Koefisien Korelasi Spearman: {spearman_coef}")

# Interpretasi
if abs(spearman_coef) < 0.3:
    strength = "lemah"
elif abs(spearman_coef) < 0.7:
    strength = "sedang"
else:
    strength = "kuat"

direction = "positif" if spearman_coef > 0 else "negatif"

print(f"Korelasi {strength} dan {direction}")

Koefisien Korelasi Spearman: -0.12998984131880323
Korelasi lemah dan negatif


# Nomor 8
- Dari beberapa Regionname manakah Regionname yang variabel jumlah jarak ke CBD (Distance) paling tidak berkorelasi dengan variabel harga rumah (Price)! 
- Serta tampilkan juga Regionname yang paling berkorelasi kedua variabel ini!
- Diasumsikan telah memenuhi syarat statistik parametrik.


In [56]:
# Korelasi Distance vs Price untuk setiap region
correlations = {}
for region in melb['Regionname'].unique():
    region_data = melb[melb['Regionname'] == region]
    if len(region_data) > 2:
        corr, _ = pearsonr(region_data['Distance'], region_data['Price'])
        correlations[region] = corr

# Region paling tidak berkorelasi (nilai absolut terkecil)
least_corr_region = min(correlations, key=lambda x: abs(correlations[x]))
least_corr_value = correlations[least_corr_region]

# Region paling berkorelasi (nilai absolut terbesar)
most_corr_region = max(correlations, key=lambda x: abs(correlations[x]))
most_corr_value = correlations[most_corr_region]

print(f"Region paling tidak berkorelasi: {least_corr_region}")
print(f"Koefisien korelasi: {least_corr_value}")
print(f"\nRegion paling berkorelasi: {most_corr_region}")
print(f"Koefisien korelasi: {most_corr_value}")

Region paling tidak berkorelasi: Southern Metropolitan
Koefisien korelasi: 0.002489149218933443

Region paling berkorelasi: Northern Metropolitan
Koefisien korelasi: -0.368551996179051


# Nomor 9
Uji hipotesis apakah metode pembelian/Method berkaitan dengan tipe rumah/ Type (dependen) atau tidak saling berkaitan (independen)!

Hypothesis:
- Ho : The two variables are independent
- Ha : The two variables are dependent.


In [57]:
# Buat contingency table
contingency_table = pd.crosstab(melb['Method'], melb['Type'])
contingency_table

# Lakukan Chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"\nChi-square statistic: {chi2}")
print(f"P-value: {p_value}")
print(f"Degrees of freedom: {dof}")

# Kesimpulan
alpha = 0.05
if p_value < alpha:
    print(f"\nKesimpulan: Tolak Ho (p-value {p_value} < α {alpha})")
    print("Kedua variabel Method dan Type saling berkaitan (dependent)")
else:
    print(f"\nKesimpulan: Gagal tolak Ho (p-value {p_value} >= α {alpha})")
    print("Kedua variabel Method dan Type tidak saling berkaitan (independent)")


Chi-square statistic: 120.08519425521135
P-value: 3.183702485154338e-22
Degrees of freedom: 8

Kesimpulan: Tolak Ho (p-value 3.183702485154338e-22 < α 0.05)
Kedua variabel Method dan Type saling berkaitan (dependent)


# Nomor 10
Uji hipotesis apakah rata-rata harga rumah (Price) antar Regionname berbeda ataukah sama! Diasumsikan persyaratan statistik parametrik terpenuhi

Hipotesis:
- Ho : μ1 = μ2 = … = μk (Rata-rata harga rumah antar Regionname sama)
- Ha : Setidaknya ada satu pasang Regionname yang rata-rata harga rumahnya tidak sama, tapi kita tidak tahu yang mana


In [58]:
regions = melb['Regionname'].unique()
price_by_region = [melb[melb['Regionname'] == region]['Price'].values for region in regions]

# Lakukan One-way ANOVA
f_stat, p_value = f_oneway(*price_by_region)

print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Kesimpulan
alpha = 0.05
if p_value < alpha:
    print(f"\nKesimpulan: Tolak Ho (p-value {p_value:.4f} < α {alpha})")
    print("Rata-rata harga rumah antar Regionname berbeda secara signifikan")
else:
    print(f"\nKesimpulan: Gagal tolak Ho (p-value {p_value:.4f} >= α {alpha})")
    print("Rata-rata harga rumah antar Regionname tidak berbeda secara signifikan")

F-statistic: 284.4002
P-value: 0.0000

Kesimpulan: Tolak Ho (p-value 0.0000 < α 0.05)
Rata-rata harga rumah antar Regionname berbeda secara signifikan
